# Introduction: Predicting Island Friendly Cars and Spy Drones with XGBoost
_**Using Gradient Boosted Trees to Predict Cars that low impact to the Island's fragile ecosystem**_

_**DIY Note:**_ We will be using this same notebook for the DIY portion of the lab by making small changes to the files and referenced frames.



---

---

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)
1. [Compile](#Compile)
1. [Host](#Host)
  1. [Evaluate](#Evaluate)
  1. [Relative cost of errors](#Relative-cost-of-errors)
1. [Extensions](#Extensions)

---

## Background

_This notebook has been adapted from an [AWS blog post](https://aws.amazon.com/blogs/ai/predicting-customer-churn-with-amazon-machine-learning/)_

Island ecosystems are fragile and to reduce the impact that some automobiles have we need a way to predict which automobiles are island friendly.  By giving island citizens the ability to input their automobile data we can give them the opportunity to steer away from automobiles that may not be healthy to the environment, cause crowding, and deplete resources. This notebook describes using machine learning (ML) for the automated identification of island friendly vehicles. ML models rarely give perfect predictions though, so this notebook is also about how to incorporate the relative costs of prediction mistakes when determining the financial outcome of using ML.

---

## Setup

_This notebook was created and tested on a `ml.m4.xlarge` notebook instance._

Let's start by updating the required packages i.e. SageMaker Python SDK, `pandas` and `numpy`, and specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the boto regexp with the appropriate full IAM role arn string(s).

#### Code Cell 1 ####

In [ ]:
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install sagemaker pandas numpy matplotlib --upgrade


#### Code Cell 2 ####

In [ ]:
import sagemaker
import boto3
import re
from sagemaker import get_execution_role

sess = sagemaker.Session()
bucket_prefix = "spy-drones-lab"

# Define lab bucket
s3 = boto3.client('s3')
for bucket in s3.list_buckets()['Buckets']:
    if bucket_prefix in bucket.get('Name'):
        bucket = bucket.get('Name')

print("default-bucket: " + bucket)
prefix = "lab/xgboost-island-friendly"

# Define IAM role
role = get_execution_role()
print("Default IAM role: " + role)

Next, we'll import the Python libraries we'll need for the remainder of the exercise.

#### Code Cell 3 ####

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import os
import sys
import time
import json
from IPython.display import display
from time import strftime, gmtime
from sagemaker.inputs import TrainingInput
from sagemaker.serializers import CSVSerializer

---
## Data

The data for this model comes from [University of California Irvine](https://archive.ics.uci.edu/ml/datasets/automobile/) Machine Learning Repository and originally used for the purposes of assigning insurance risk scores to vehicles.  We have modified the data set slightly for our purposes.  We can use this historical information to construct an ML model to predict island friendly vehicles using a process called training. After training the model, we can pass the profile information of an arbitrary car (the same profile information that we used to train the model) to the model, and have the model predict whether this vehicle is island friendly or not.  Of course, we expect the model to make mistakes–after all, predicting the future is tricky business! But I’ll also show how to deal with prediction errors.

Let's download and read that dataset in now:

**DIY Note:** Comment out the line for download of IslandAutos.csv and uncomment the line for DroneFlights.csv 

#### Code Cell 4 ####

In [ ]:
## For DIY section comment out the below line
!aws s3 cp s3://{bucket}/IslandAutos.csv ./
    
## For DIY section uncomment the below line
#!aws s3 cp s3://{bucket}/DroneFlights.csv ./
    

### Working with missing data

The data scientist that provided the data did some clean-up and any previously empty or erroneous values have been populated with a question mark (?).  We will import the data using the na_values parameter that references a list of known missing values.  

As you work through the data and see other types of missing values, you can add them to the list below:

**DIY Note:** In the below code cell change `IslandAutos.csv` file to `DroneFlights.csv` 

#### Code Cell 5 ####

In [ ]:
# Make a list of missing value types
missing_values = ["?"]

ml_data = pd.read_csv("./IslandAutos.csv", na_values = missing_values)
pd.set_option("display.max_columns", 500)

#display a summary view
ml_data

### Dealing with missing data

The imported data frame above has 205 rows. The below code creates a new dataframe and prints out all rows with a least one attribute marked as __NaN__.  As you can see only a few rows have empty values.

**DIY Note:** Some of the earlier drone models (those with id less then 100) had bad transponders that malfunctioned with speeds exceeding safety standards.  We will clean those up in the next few steps/

#### Code Cell 6 ####

In [ ]:
#New Data Frame that displays all rows with at least one attribute marked NaN
missing_df = ml_data[ml_data.isna().any(axis=1)]
missing_df


#### Code Cell 7 ####

In [ ]:
## Fill the missing data and or delete bad data.
## The next three lines need to be commnented out for the DIY portion of the lab.  

mean = ml_data['price'].mean()                         #Comment out this line for DIY portion of lab
print("The mean vehicle price is: " + str(mean))       #Comment out this line for DIY portion of lab
ml_data['price'].fillna(mean)                          #Comment out this line for DIY portion of lab

#Drop any rows which still have missing values
ml_data.dropna(axis=0, how='any',inplace=True)

#Summary view - Notice the # of data values was reduced after dropping data.
ml_data


### Data Features ###

Now that we have done a first pass at cleaning up the data lets review more details about it.  

By modern standards, it’s a relatively small dataset, with less than 200 records, where each record has 16 attributes to describe the profile of a vehicle. The attributes are:

- `make`: The automobile manufacturer; for example, audi or dodge.
- `fuel-type`: Diesel or Gas.
- `aspiration`: turbo vs standard.
- `num-of-doors`: Number of doors (Two or Four).
- `body-style`: wagon, sedan, hatchback, etc. 
- `drive-wheels`: front wheel drive (fwd), four wheel drive (4wd) or rear-wheel drive (rwd)
- `engine-location`: Front or Back.
- `curb-weight`: continuous from 1488 to 4066
- `engine-type`: dohc, dohcv, l, ohc, ohcf, ohcv, rotor.
- `num-of-cylinders`: eight, five, four, six, three, twelve, two.
- `horsepower`: continuous from 48 to 288.
- `peak-rpm`: continuous from 4150 to 6600.
- `city-mpg`: continuous from 13 to 49.
- `highway-mpg`: continuous from 16 to 54.
- `price`: continuous from 5118 to 45400.
- `island-friendly?`: 0 = no; 1 = yes.


The last attribute, `island-friendly?`, is known as the target attribute–the attribute that we want the ML model to predict.  Because the target attribute is binary, our model will be performing binary prediction, also known as binary classification.

Let's begin exploring the data:


**DIY Note:** The target attribute for the DroneFLights data set is `spy?` 

Other fields in the DroneFlights data set are:
- `drone-id`: Randomly generated id for drones.  Older drones have lower id numbers.
- `flight-duration`: Friendly drones normally do not fly for more than 20 minutes. 
- `transponder`: On (1) or Off (0); Older spy drones did not have transponders; but newere ones can fake a transponder signal.
- `max-altitide`: Spy drones fly at higher altitudes. 
- `max-speed`: Spy drones display significantly higher speeds since when spotted they need to make a quick escape.


#### Code Cell 8 ####

In [ ]:
for column in ml_data.select_dtypes(include=["object"]).columns:
    display(pd.crosstab(index=ml_data[column], columns="% observations", normalize="columns"))

# Histograms for each numeric features
display(ml_data.describe())
%matplotlib inline
hist = ml_data.hist(bins=30, sharey=True, figsize=(10, 10))

We can see immediately that:
- `city-mpg` is quite quite evenly distributed
- `highway-mpg` statistics are very similar to city-mpg.  So we will drop highway-mpg as it seems redundant.
- Most of the numeric features are surprisingly nicely distributed, with many showing bell-like `gaussianity`.  `peak-rpm` being a notable exception, so we will drop that feature from the data as well.


**DIY Note:** We can see that `drone-id` is evenly distributed and there is no correlation between it and a drone being friendly or a spy.  So we will drop that feature from our data set.

#### Code Cell 9 ####

In [ ]:
# DIY Note:  You can skip running of the next two lines by commenting them out. 

ml_data = ml_data.drop("highway-mpg", axis=1)
ml_data = ml_data.drop("peak-rpm", axis=1)

In [ ]:
# DIY Note:  Uncomment the next line to drop the feature `drone-id` from the data set. 
#ml_data = ml_data.drop("drone-id", axis=1)

Next let's look at the relationship between each of the features and our target variable.

**DIY Note:** In the below code cells carefully review and replace reference `island-friendly?` with `spy?`

#### Code Cell 10 ####

In [ ]:
for column in ml_data.select_dtypes(include=["object"]).columns:
    if column != "island-friendly?":
        display(pd.crosstab(index=ml_data[column], columns=ml_data["island-friendly?"], normalize="columns"))

for column in ml_data.select_dtypes(exclude=["object"]).columns:
    if column != "island-friendly?":
        print(column)
        hist = ml_data[[column, "island-friendly?"]].hist(by="island-friendly?", bins=30)
        plt.show()

#### Code Cell 11 ####

In [ ]:
display(ml_data.corr(numeric_only=True))
pd.plotting.scatter_matrix(ml_data, figsize=(14, 14))
plt.show()

#### Code Cell 12 ####

In [ ]:
# Final dimensions of the dataset
ml_data.shape

Now that we've cleaned up our dataset, let's determine which algorithm to use.  As mentioned above, there appears to be some variables where both high and low (but not intermediate) values are predictive of non island-friendly vehicles. In order to accommodate this in an algorithm like linear regression, we'd need to generate polynomial (or bucketed) terms.  Instead, let's attempt to model this problem using gradient boosted trees.  Amazon SageMaker provides an XGBoost container that we can use to train in a managed, distributed setting, and then host as a real-time prediction endpoint.  XGBoost uses gradient boosted trees which naturally account for non-linear relationships between features and the target variable, as well as accommodating complex interactions between features.

Amazon SageMaker XGBoost can train on data in either a CSV or LibSVM format.  For this example, we'll stick with CSV.  It should:
- Have the predictor variable in the first column
- Not have a header row

But first, let's convert our categorical features into numeric features.

#### Code Cell 13 ####

In [ ]:
model_data = pd.get_dummies(ml_data,dtype=float)

model_data

And now let's split the data into training, validation, and test sets.  This will help prevent us from over-fitting the model, and allow us to test the model's accuracy on data it hasn't already seen.

#### Code Cell 14 ####

In [ ]:
train_data, validation_data, test_data = np.split(
    model_data.sample(frac=1, random_state=1729),
    [int(0.7 * len(model_data)), int(0.9 * len(model_data))],
)
train_data.to_csv("train.csv", header=False, index=False)
validation_data.to_csv("validation.csv", header=False, index=False)

Now we'll upload these files to S3.

#### Code Cell 15 ####

In [ ]:
boto3.Session().resource("s3").Bucket(bucket).Object(
    os.path.join(prefix, "train/train.csv")
).upload_file("train.csv")
boto3.Session().resource("s3").Bucket(bucket).Object(
    os.path.join(prefix, "validation/validation.csv")
).upload_file("validation.csv")

---
## Train

Moving onto training, first we'll need to specify the locations of the XGBoost algorithm containers.

#### Code Cell 16 ####

In [ ]:
container = sagemaker.image_uris.retrieve("xgboost", boto3.Session().region_name, "latest")
display(container)

Then, because we're training with the CSV file format, we'll create `TrainingInput`s that our training function can use as a pointer to the files in S3.

#### Code Cell 17 ####

In [ ]:
s3_input_train = TrainingInput(
    s3_data="s3://{}/{}/train".format(bucket, prefix), content_type="csv"
)
s3_input_validation = TrainingInput(
    s3_data="s3://{}/{}/validation/".format(bucket, prefix), content_type="csv"
)

Now, we can specify a few parameters like what type of training instances we'd like to use and how many, as well as our XGBoost hyperparameters.  A few key hyperparameters are:
- `max_depth` controls how deep each tree within the algorithm can be built.  Deeper trees can lead to better fit, but are more computationally expensive and can lead to overfitting.  There is typically some trade-off in model performance that needs to be explored between numerous shallow trees and a smaller number of deeper trees.
- `subsample` controls sampling of the training data.  This technique can help reduce overfitting, but setting it too low can also starve the model of data.
- `num_round` controls the number of boosting rounds.  This is essentially the subsequent models that are trained using the residuals of previous iterations.  Again, more rounds should produce a better fit on the training data, but can be computationally expensive or lead to over-fitting.
- `eta` controls how aggressive each round of boosting is.  Larger values lead to more conservative boosting.
- `gamma` controls how aggressively trees are grown.  Larger values lead to more conservative models.

More detail on XGBoost's hyper-parameters can be found on their GitHub [page](https://github.com/dmlc/xgboost/blob/master/doc/parameter.md).

#### Code Cell 18 ####

In [ ]:
sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(
    container,
    role,
    instance_count=1,
    instance_type="ml.m4.xlarge",
    output_path="s3://{}/{}/output".format(bucket, prefix),
    sagemaker_session=sess,
)
xgb.set_hyperparameters(
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.8,
    silent=0,
    objective="binary:logistic",
    num_round=100,
)

xgb.fit({"train": s3_input_train, "validation": s3_input_validation})

---
## Host

Now that we've trained the algorithm, let's create a model and deploy it to a hosted endpoint.

#### Code Cell 19 ####

In [ ]:
xgb_predictor = xgb.deploy(
    initial_instance_count=1, instance_type="ml.m4.xlarge", serializer=CSVSerializer()
)

### Evaluate

Now that we have a hosted endpoint running, we can make real-time predictions from our model very easily, simply by making a `http` POST request.  But first, we'll need to set up serializers and deserializers for passing our `test_data` NumPy arrays to the model behind the endpoint.

Now, we'll use a simple function to:
1. Loop over our test dataset
1. Split it into mini-batches of rows 
1. Convert those mini-batchs to CSV string payloads
1. Retrieve mini-batch predictions by invoking the XGBoost endpoint
1. Collect predictions and convert from the CSV output our model provides into a NumPy array

#### Code Cell 20 ####

In [ ]:
def predict(data, rows=50):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ""
    for array in split_array:
        predictions = ",".join([predictions, xgb_predictor.predict(array).decode("utf-8")])

    return np.fromstring(predictions[1:], sep=",")


predictions = predict(test_data.to_numpy()[:, 1:])

#### Code Cell 21 ####

In [ ]:
print(test_data)
print(predictions)

There are many ways to compare the performance of a machine learning model, but let's start by simply by comparing actual to predicted values.  In this case, we're simply predicting whether the vehicle is island friendly (`1`) or not (`0`), which produces a simple confusion matrix.

#### Code Cell 22 ####

In [ ]:
pd.crosstab(
    index=test_data.iloc[:, 0],
    columns=np.round(predictions),
    rownames=["actual"],
    colnames=["predictions"],
)

_Note, due to randomized elements of the algorithm, your results may differ slightly._

Of the 20 vehicles, we've correctly predicted 19 of them (true positives). And, we incorrectly predicted 1 vehicle to be island-friendly but ended up not being so which is known as a false positive.  There are no false negatives in our testing.  

An important point here is that because of the `np.round()` function above we are using a simple threshold (or cutoff) of 0.5.  Our predictions from `xgboost` come out as continuous values between 0 and 1, and we force them into the binary classes that we began with.  
To get a rough intuition here, let's look at the continuous values of our predictions.

#### Code Cell 23 ####

In [ ]:
plt.hist(predictions)
plt.show()